# Task 4 Exploration

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, callbacks, metrics, regularizers
from tensorflow.keras.layers import Concatenate, Input

def load_and_combine_data(filenames):
    data_frames = []
    for file in filenames:
        df = pd.read_csv(file, sep='\t')
        data_frames.append(df)
    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

def train_val_split(df, type_col='Type', target_col='GenreID'):
    train_data = df[df[type_col] == 'Train']
    test_data = df[df[type_col] == 'Test']
    X_train = train_data.drop([type_col, target_col], axis=1)
    y_train = train_data[target_col]
    X_test = test_data.drop([type_col, target_col], axis=1)
    y_test = test_data[target_col]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, y_train, X_test_scaled, y_test

Step 1: choose which dataset to include (include path)

In [45]:
data_paths = [
    # '../Classification Music/GenreClassData_10s.txt',
    '../Classification Music/GenreClassData_30s.txt',
    # '../Classification Music/GenreClassData_5s.txt'
] 

Step 2: load and prepare data

In [46]:
df = load_and_combine_data(data_paths)
feature_columns = [col for col in df.columns if col not in ['Type', 'GenreID', 'Genre', 'Track ID', 'File']]

df = df[['Type', 'GenreID'] + feature_columns] # keeping only relevent columns

print("Combined Dataframe shape:", df.shape) 

X_train_scaled, y_train, X_test_scaled, y_test = train_val_split(df)

Combined Dataframe shape: (990, 65)


# 1. Fully Connected Neural Network (FCNN) Section

### 1.1 Create FCNN model

In [47]:
fcnn = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])
fcnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


C:\Users\menbu\PycharmProjects\Music Genre Classification\venv\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### 1.1.1 Alternative FCNN architectures

### Deep and Wide FCNN

In [48]:
deep_wide_fcnn = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax')
])
deep_wide_fcnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Shallow Network

In [49]:
shallow_fcnn = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])
shallow_fcnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### FCNN with Batch Normalization

In [50]:
bn_fcnn = models.Sequential([
    layers.Dense(128, input_shape=(X_train_scaled.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dense(128),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dense(128),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])
bn_fcnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### FCNN with Alternative Activation Functions

In [51]:
alt_act_fcnn = models.Sequential([
    layers.Dense(128, activation='elu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(128, activation='elu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='elu'),
    layers.Dense(10, activation='softmax')
])
alt_act_fcnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### FCNN with L1/L2 Regularization

In [52]:
reg_fcnn = models.Sequential([
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(10, activation='softmax')
])
reg_fcnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### 1.2 Train FCNN

simply change the "model" parameter to change the model to train

In [53]:
model = deep_wide_fcnn # name of the model to be trained here

history_fcnn = model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled, y_test))
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"FCNN Test Accuracy: {test_acc:.4f}")

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.2271 - loss: 2.1411 - val_accuracy: 0.5000 - val_loss: 1.4320
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4816 - loss: 1.3993 - val_accuracy: 0.5909 - val_loss: 1.1206
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6093 - loss: 1.0460 - val_accuracy: 0.6717 - val_loss: 0.9878
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7183 - loss: 0.8534 - val_accuracy: 0.7020 - val_loss: 0.9449
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8039 - loss: 0.6407 - val_accuracy: 0.7071 - val_loss: 0.8621
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8535 - loss: 0.5136 - val_accuracy: 0.7222 - val_loss: 0.9003
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8450 - loss: 0.4368 - val_accuracy: 0.7071 - val_loss: 0.9702
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8704 - loss: 0.3596 - val_accuracy: 0.7121 - val_loss:

# 2. Convolutional Neural Network (CNN) Section

### 2.1 Reshape data for CNN

In [54]:
X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)


### 2.2 Create CNN model

In [55]:
cnn = models.Sequential([
    layers.Conv1D(32, 3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    layers.MaxPooling1D(2),
    layers.Conv1D(32, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


C:\Users\menbu\PycharmProjects\Music Genre Classification\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


### 2.2.1 Alternative CNN architectures

### Increased Depth and Filters

In [56]:
deep_cnn = models.Sequential([
    layers.Conv1D(64, 3, activation='relu', padding='same', input_shape=(X_train_scaled.shape[1], 1)),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
deep_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### CNN with Batch Normalization

In [57]:
bn_cnn = models.Sequential([
    layers.Conv1D(32, 3, padding='same', input_shape=(X_train_scaled.shape[1], 1)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])
bn_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### CNN with Dilated Convolutions

In [58]:
dilated_cnn = models.Sequential([
    layers.Conv1D(32, 3, dilation_rate=1, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    layers.Conv1D(32, 3, dilation_rate=2, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 3, dilation_rate=2, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])
dilated_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### CNN with Inception Modules

In [59]:
def inception_module(input_tensor, filter_operation):
    t1 = layers.Conv1D(filter_operation, 1, padding='same', activation='relu')(input_tensor)

    t2 = layers.Conv1D(filter_operation, 1, padding='same', activation='relu')(input_tensor)
    t2 = layers.Conv1D(filter_operation, 3, padding='same', activation='relu')(t2)

    t3 = layers.Conv1D(filter_operation, 1, padding='same', activation='relu')(input_tensor)
    t3 = layers.Conv1D(filter_operation, 5, padding='same', activation='relu')(t3)

    t4 = layers.MaxPooling1D(3, strides=1, padding='same')(input_tensor)
    t4 = layers.Conv1D(filter_operation, 1, padding='same', activation='relu')(t4)

    output = layers.Concatenate()([t1, t2, t3, t4])
    return output

input_layer = Input(shape=(X_train_scaled.shape[1], 1))
x = inception_module(input_layer, 32)
x = layers.MaxPooling1D(2)(x)
x = inception_module(x, 64)
x = layers.MaxPooling1D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
output_layer = layers.Dense(10, activation='softmax')(x)

inception_cnn = models.Model(inputs=input_layer, outputs=output_layer)
inception_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Lightweight CNN

In [60]:
lightweight_cnn = models.Sequential([
    layers.Conv1D(16, 3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    layers.MaxPooling1D(2),
    layers.Conv1D(16, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])
lightweight_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 2.3 Train CNN model

simply change the "model" parameter to change the model to train

In [61]:
model = bn_cnn # name of the model to be trained here

history_cnn = model.fit(X_train_cnn, y_train, epochs=10, validation_data=(X_test_cnn, y_test))
test_loss, test_acc = model.evaluate(X_test_cnn, y_test, verbose=2)
print(f"CNN Test Accuracy: {test_acc:.4f}")

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2609 - loss: 2.2391 - val_accuracy: 0.5101 - val_loss: 1.9206
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5823 - loss: 1.1847 - val_accuracy: 0.5707 - val_loss: 1.7884
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6606 - loss: 0.9673 - val_accuracy: 0.4798 - val_loss: 1.6982
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7238 - loss: 0.7945 - val_accuracy: 0.4798 - val_loss: 1.6313
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7836 - loss: 0.6847 - val_accuracy: 0.4394 - val_loss: 1.5892
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7788 - loss: 0.6043 - val_accuracy: 0.4040 - val_loss: 1.5988
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8413 - loss: 0.4941 - val_accuracy: 0.5000 - val_loss: 1.4020
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8785 - loss: 0.4290 - val_accuracy: 0.4646 - val_loss

# 3. Recurrent Neural Network (RNN) Section

### 3.1 Reshape data for RNN

In [62]:
X_train_rnn = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_rnn = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

### 3.2 Create RNN model

In [63]:
rnn = models.Sequential([
    layers.LSTM(64, return_sequences=True, input_shape=(1, X_train_scaled.shape[1])),
    layers.LSTM(64),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])
rnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

C:\Users\menbu\PycharmProjects\Music Genre Classification\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### 3.2.1 Alternative RNN architectures

### Simple RNN with Increased Depth

In [64]:
simple_rnn = models.Sequential([
    layers.SimpleRNN(64, return_sequences=True, input_shape=(1, X_train_scaled.shape[1])),
    layers.SimpleRNN(64, return_sequences=True),
    layers.SimpleRNN(64),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])
simple_rnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### LSTM with Bidirectional Layers

In [65]:
bidirectional_lstm = models.Sequential([
    layers.Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(1, X_train_scaled.shape[1])),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])
bidirectional_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

C:\Users\menbu\PycharmProjects\Music Genre Classification\venv\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### GRU with Layer Normalization

In [66]:
gru = models.Sequential([
    layers.GRU(64, return_sequences=True, input_shape=(1, X_train_scaled.shape[1]), recurrent_initializer='glorot_uniform'),
    layers.LayerNormalization(),
    layers.GRU(64),
    layers.LayerNormalization(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])
gru.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Stacked LSTM with Different Units

In [67]:
stacked_lstm = models.Sequential([
    layers.LSTM(128, return_sequences=True, input_shape=(1, X_train_scaled.shape[1])),
    layers.LSTM(64, return_sequences=True),
    layers.LSTM(32),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])
stacked_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 3.3 Train RNN

simply change the "model" parameter to change the model to train

In [68]:
model = bidirectional_lstm # name of the model to be trained here

history_rnn = model.fit(X_train_rnn, y_train, epochs=20, validation_data=(X_test_rnn, y_test))
test_loss, test_acc = model.evaluate(X_test_rnn, y_test, verbose=2)
print(f"RNN Test Accuracy: {test_acc:.4f}")

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.2388 - loss: 2.2524 - val_accuracy: 0.3788 - val_loss: 1.9818
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4056 - loss: 1.8804 - val_accuracy: 0.4192 - val_loss: 1.5380
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4527 - loss: 1.4684 - val_accuracy: 0.5455 - val_loss: 1.2638
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5551 - loss: 1.2300 - val_accuracy: 0.6010 - val_loss: 1.0931
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6384 - loss: 1.0457 - val_accuracy: 0.6313 - val_loss: 1.0004
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7211 - loss: 0.8487 - val_accuracy: 0.6717 - val_loss: 0.9510
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7511 - loss: 0.7541 - val_accuracy: 0.6616 - val_loss: 0.9219
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7818 - loss: 0.6528 - val_accuracy: 0.6970 - val_loss

# 4. Hybrid models Section

### CNN + LSTM Model (it's bad)

In [69]:
hybrid_cnn_lstm_model = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.LSTM(64, return_sequences=True),
    layers.LSTM(64),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

hybrid_cnn_lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### LSTM + Conv1D Model (slow to train and overfits)

In [70]:
hybrid_lstm_cnn_model = models.Sequential([
    layers.LSTM(64, return_sequences=True, input_shape=(X_train_scaled.shape[1], 1)),
    layers.LSTM(64),
    layers.Reshape((64, 1)),
    layers.Conv1D(64, 3, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

hybrid_lstm_cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### CNN + GRU Model (overfits)

In [71]:
hybrid_cnn_gru_model = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.GRU(64, return_sequences=True),
    layers.GRU(64),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

hybrid_cnn_gru_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Bidirectional LSTM + CNN Model (overfits)

In [72]:
hybrid_bidirectional_lstm_cnn_model = models.Sequential([
    layers.Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(X_train_scaled.shape[1], 1)),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Reshape((128, 1)),
    layers.Conv1D(64, 3, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

hybrid_bidirectional_lstm_cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### 4.1 Train hybrid models

In [73]:
model = hybrid_cnn_lstm_model # name of the model to be trained here

history_hybrid = model.fit(X_train_scaled, y_train, epochs=30, validation_data=(X_test_scaled, y_test))
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"RNN Test Accuracy: {test_acc:.4f}")

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.1471 - loss: 2.2695 - val_accuracy: 0.2071 - val_loss: 2.0771
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2117 - loss: 2.0702 - val_accuracy: 0.1970 - val_loss: 1.9754
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2663 - loss: 1.9504 - val_accuracy: 0.2828 - val_loss: 1.8649
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2896 - loss: 1.8258 - val_accuracy: 0.3889 - val_loss: 1.7679
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3218 - loss: 1.7448 - val_accuracy: 0.3737 - val_loss: 1.7233
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3510 - loss: 1.7226 - val_accuracy: 0.3636 - val_loss: 1.6863
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3580 - loss: 1.7389 - val_accuracy: 0.3434 - val_loss: 1.7199
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3152 - loss: 1.7536 - val_accuracy: 0.3636 - v

# 5. All Models

In [74]:
# List of all models to be trained with their corresponding data reshaping requirements
models_to_train = [
    {'model': fcnn, 'data': (X_train_scaled, X_test_scaled), 'name': 'Fully Connected Neural Network'},
    {'model': deep_wide_fcnn, 'data': (X_train_scaled, X_test_scaled), 'name': 'Deep and Wide FCNN'},
    {'model': shallow_fcnn, 'data': (X_train_scaled, X_test_scaled), 'name': 'Shallow FCNN'},
    {'model': bn_fcnn, 'data': (X_train_scaled, X_test_scaled), 'name': 'Batch Normalized FCNN'},
    {'model': alt_act_fcnn, 'data': (X_train_scaled, X_test_scaled), 'name': 'FCNN with Alternative Activations'},
    {'model': reg_fcnn, 'data': (X_train_scaled, X_test_scaled), 'name': 'Regularized FCNN'},
    {'model': cnn, 'data': (X_train_cnn, X_test_cnn), 'name': 'Basic CNN'},
    {'model': deep_cnn, 'data': (X_train_cnn, X_test_cnn), 'name': 'Deep CNN'},
    {'model': bn_cnn, 'data': (X_train_cnn, X_test_cnn), 'name': 'Batch Normalized CNN'},
    {'model': dilated_cnn, 'data': (X_train_cnn, X_test_cnn), 'name': 'Dilated CNN'},
    {'model': inception_cnn, 'data': (X_train_cnn, X_test_cnn), 'name': 'Inception CNN'},
    {'model': lightweight_cnn, 'data': (X_train_cnn, X_test_cnn), 'name': 'Lightweight CNN'},
    {'model': rnn, 'data': (X_train_rnn, X_test_rnn), 'name': 'Basic RNN'},
    {'model': simple_rnn, 'data': (X_train_rnn, X_test_rnn), 'name': 'Simple RNN with Increased Depth'},
    {'model': bidirectional_lstm, 'data': (X_train_rnn, X_test_rnn), 'name': 'Bidirectional LSTM'},
    {'model': gru, 'data': (X_train_rnn, X_test_rnn), 'name': 'GRU with Layer Normalization'},
    {'model': stacked_lstm, 'data': (X_train_rnn, X_test_rnn), 'name': 'Stacked LSTM'},
    {'model': hybrid_cnn_lstm_model, 'data': (X_train_scaled, X_test_scaled), 'name': 'Hybrid CNN-LSTM'},
    {'model': hybrid_lstm_cnn_model, 'data': (X_train_scaled, X_test_scaled), 'name': 'Hybrid LSTM-CNN'},
    {'model': hybrid_cnn_gru_model, 'data': (X_train_scaled, X_test_scaled), 'name': 'Hybrid CNN-GRU'},
    {'model': hybrid_bidirectional_lstm_cnn_model, 'data': (X_train_scaled, X_test_scaled), 'name': 'Hybrid Bidirectional LSTM-CNN'}
]

results = []

# Training and evaluating each model
for entry in models_to_train:
    model = entry['model']
    X_train, X_test = entry['data']
    y_train, y_test = y_train, y_test  # Assuming y_train and y_test are defined globally

    print(f"Training {entry['name']}...")
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=0)
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"{entry['name']} Test Accuracy: {test_acc:.4f}")

    results.append({'model': entry['name'], 'accuracy': test_acc})

# Sorting results by accuracy
sorted_results = sorted(results, key=lambda x: x['accuracy'], reverse=True)

# Displaying sorted results
print("Models ranked by test accuracy:")
for result in sorted_results:
    print(f"{result['model']}: {result['accuracy']:.4f}")


Training Fully Connected Neural Network...
Fully Connected Neural Network Test Accuracy: 0.7121
Training Deep and Wide FCNN...
Deep and Wide FCNN Test Accuracy: 0.7323
Training Shallow FCNN...
Shallow FCNN Test Accuracy: 0.6667
Training Batch Normalized FCNN...
Batch Normalized FCNN Test Accuracy: 0.7475
Training FCNN with Alternative Activations...
FCNN with Alternative Activations Test Accuracy: 0.7323
Training Regularized FCNN...
Regularized FCNN Test Accuracy: 0.6869
Training Basic CNN...
Basic CNN Test Accuracy: 0.6869
Training Deep CNN...
Deep CNN Test Accuracy: 0.5960
Training Batch Normalized CNN...
Batch Normalized CNN Test Accuracy: 0.7222
Training Dilated CNN...
Dilated CNN Test Accuracy: 0.7071
Training Inception CNN...
Inception CNN Test Accuracy: 0.7121
Training Lightweight CNN...
Lightweight CNN Test Accuracy: 0.6818
Training Basic RNN...
Basic RNN Test Accuracy: 0.6869
Training Simple RNN with Increased Depth...
Simple RNN with Increased Depth Test Accuracy: 0.7020
Trai